In [ ]:
# 1) Split board into 64 segments
# 2) Infrence upon each image
# 3) Results pushed to images and text
# 4) Push results into array, text file is closed

In [1]:
# installs dependencies, restart kernel afterwards
%pip install -r requirements_CPU.txt


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [94]:
# import image slicer library and segment original image into 64 seperate peices, ideally the 64 boxes on the chess board
# does not need to be ran every time once splits are created
import image_slicer
image_slicer.slice('chessboard_splits/chessboard.jpg', 64)

(<Tile #1 - chessboard_01_01.png>,
 <Tile #2 - chessboard_01_02.png>,
 <Tile #3 - chessboard_01_03.png>,
 <Tile #4 - chessboard_01_04.png>,
 <Tile #5 - chessboard_01_05.png>,
 <Tile #6 - chessboard_01_06.png>,
 <Tile #7 - chessboard_01_07.png>,
 <Tile #8 - chessboard_01_08.png>,
 <Tile #9 - chessboard_02_01.png>,
 <Tile #10 - chessboard_02_02.png>,
 <Tile #11 - chessboard_02_03.png>,
 <Tile #12 - chessboard_02_04.png>,
 <Tile #13 - chessboard_02_05.png>,
 <Tile #14 - chessboard_02_06.png>,
 <Tile #15 - chessboard_02_07.png>,
 <Tile #16 - chessboard_02_08.png>,
 <Tile #17 - chessboard_03_01.png>,
 <Tile #18 - chessboard_03_02.png>,
 <Tile #19 - chessboard_03_03.png>,
 <Tile #20 - chessboard_03_04.png>,
 <Tile #21 - chessboard_03_05.png>,
 <Tile #22 - chessboard_03_06.png>,
 <Tile #23 - chessboard_03_07.png>,
 <Tile #24 - chessboard_03_08.png>,
 <Tile #25 - chessboard_04_01.png>,
 <Tile #26 - chessboard_04_02.png>,
 <Tile #27 - chessboard_04_03.png>,
 <Tile #28 - chessboard_04_04.png>,
 

In [95]:
# libpng will compain about sRGB profiles so it is neccesary to use imageMagicks mogrify to remove the iCCP chunk from the split images.
# Binaries for Linux/MacOS and Windows are included in the "magick" directory. ImageMagick is REQUIRED to run this block. See here for details: https://imagemagick.org/script/download.php#linux
# It IS possible to continue without running this block.

# vvv UNCOMMENT BELOW FOR INSTALLING IMAGEMAGICK ON MACOS vvv

#!brew install imagemagick
#!brew install ghostscript

# For Linux and Windows, install ImageMagick with the executables in the "magick" directory

!mogrify chessboard_splits/*.png

14238.79s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [104]:
# running model on each image created by image_slicer
import os
from PIL import Image
from os import listdir
from roboflow import Roboflow
rf = Roboflow(api_key="0uya8edueMEE3249OloG")
project = rf.workspace().project("piece-detection-rayven")
model = project.version(4).model

text_file = open("classes.txt", "w")

path = "/Users/connorscally/Documents/GitHub/ChessVision/BoardDigitizer/chessboard_splits/"

for image in os.listdir(path):
    image = path + image
    prediction = (model.predict(image, confidence=40, overlap=30).json())
    eval = prediction["predictions"][0]["class"]
    print(prediction["predictions"])
    # writes class names assigned during image evaluation to a text file as long as class names are a chess piece, if not, 0 is written instead
    
    if eval=='background' or eval=='Background' or eval =='':
        text_file.write('0')
        text_file.write('\n')
    
    else:
        text_file.write(eval)
        text_file.write('\n')

text_file.close()

loading Roboflow workspace...
loading Roboflow project...
[{'x': 180.5, 'y': 267.5, 'width': 271.0, 'height': 511.0, 'confidence': 0.9011098146438599, 'class': 'White-Queen', 'image_path': '/Users/connorscally/Documents/GitHub/ChessVision/BoardDigitizer/chessboard_splits/chessboard.jpg', 'prediction_type': 'ObjectDetectionModel'}]


In [105]:
import numpy as np

# Text file data converted to string data type
piece_array = np.loadtxt("classes.txt", dtype=str)
print(piece_array)

White-Queen
